In [ ]:
# Parameters: 
affiliation = 'CSU'
aggregation = 'week' # aggregation can be by day ('day'), week ('week') or month('month')
from_date <-'2017-09-01'
to_date <-'2018-12-31'

In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

## Using API

In [ ]:
base <- "193.175.238.88:8080/api/politicians/"
call1 <- paste(base,"all/", sep="")
get_politicians_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
party_df <- get_politicians_df[get_politicians_df$affiliation == affiliation & !is.null(get_politicians_df$wp_ids) ,]

In [ ]:
base_ww <- "https://www.wikiwho.net/de/edit_persistence/v1.0.0-beta/page/"
base_ch <- paste(base, "wikipedia/chobs/politicians/page_id/",sep="")
chobs <- data.frame()
wikiwho <- data.frame()
for (wp_id in party_df$wp_ids) {
    for (wp in wp_id) {
        call_chobs <- paste(base_ch, toString(wp), "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
        json_chobs <- fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"))
        if (length(json_chobs$values) != 0)  {
            chobs_unique <- as.data.frame(fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"), flatten = TRUE))
            chobs_unique$wp_user_id = wp
            chobs <- rbind(chobs, chobs_unique)
        }
        call_wikiwho <- paste(base_ww, wp ,"/?start=",from_date,"&end=",to_date,sep="")
        json_wikiwho <- fromJSON(content(GET(call_wikiwho), "text", encoding="UTF-8"))
        wikiwho <- rbind(wikiwho, json_wikiwho$editions)
    }
}

wikiwho$undos <- wikiwho$dels + wikiwho$reins
wikiwho <- summarise(group_by(wikiwho, year_month), undos = sum(undos),revisions = sum(revisions), conflicts = sum(conflicts))
chobs <- summarise(group_by(chobs, labels), values = sum(values))

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = chobs, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = values, color="Change objects")) + 
  labs(title = "Wikipedia Activity", x = "", y = "Change Objects") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), legend.position="none") +
  scale_x_date(date_breaks = "1 month")+ scale_y_continuous(labels = scales::number_format(big.mark = ""))

options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = wikiwho, mapping = aes(as.Date(year_month))) +
  geom_line(aes(y = revisions, color="Revisions"))  + 
  labs(title = "", x = "Date", y = "Revisions") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), legend.position="none") +
  scale_x_date(date_breaks = "1 month")+ scale_y_continuous(labels = scales::number_format(big.mark = ""))

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = wikiwho, mapping = aes(as.Date(year_month))) +
  geom_line(aes(y = undos, color="Undos")) +
  geom_line(aes(y = conflicts, color="Conflicts")) +
  labs(title = "Wikipedia Conflicts", x = "Date", y = "Undos and Conflicts") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month") + scale_y_continuous(labels = scales::number_format(big.mark = ""))